In [2]:
import os, sys

module_path = os.path.abspath('../..')
if module_path not in sys.path:
    sys.path.append(module_path)
    
import numpy as np    
import networkx as nx
from unisys import gate, circuit
from unisys import Gate, Circuit
from unisys.utils import operator, arch


In [3]:
inserted_swap = gate.SWAP.on([0,3])
circ = circuit.Circuit([
    gate.H.on(0), gate.H.on(2), gate.H.on(5),
    gate.Z.on(0), gate.X.on(2, 1), gate.X.on(5, 4),
    gate.X.on(1, 0), gate.X.on(3, 2),
    gate.H.on(2), gate.H.on(3),
    gate.X.on(2, 1), gate.X.on(5, 3),
    gate.Z.on(3),
    gate.X.on(3, 4),
    gate.X.on(0, 3)
])
circ_with_swaps = circ.clone()
circ_with_swaps.insert(-2, inserted_swap)

In [4]:
print(circ_with_swaps)

Circuit(num_gates: 16, num_qubits: 6, with_measure: False)


In [8]:
device = arch.read_device_topology('../../benchmark/topology/oslo.graphml')
mapping = arch.gene_init_mapping(circ, device)
updated_mapping = arch.update_mapping(mapping, inserted_swap)
print('Initial mapping:', mapping)
print('Updated mapping: {} after inserting {}'.format(updated_mapping, inserted_swap))


--------------------------------------------------
logical_qubits: [0, 1, 2, 3, 4, 5]
physical_qubits: [0, 1, 2, 3, 4, 5, 6]
Initial mapping: {0: 1, 1: 6, 2: 4, 3: 3, 4: 5, 5: 0}
Updated mapping: {0: 3, 1: 6, 2: 4, 3: 1, 4: 5, 5: 0} after inserting SWAP: targ [0, 3]


In [9]:
arch.obtain_logic_logic_mapping(mapping, updated_mapping)

{0: 3, 1: 1, 2: 2, 3: 0, 4: 4, 5: 5}

In [10]:
print(circ.gates)

[H: targ [0], H: targ [2], H: targ [5], Z: targ [0], X: targ [2] | ctrl [1], X: targ [5] | ctrl [4], X: targ [1] | ctrl [0], X: targ [3] | ctrl [2], H: targ [2], H: targ [3], X: targ [2] | ctrl [1], X: targ [5] | ctrl [3], Z: targ [3], X: targ [3] | ctrl [4], X: targ [0] | ctrl [3]]


In [11]:
print(circ)
print(circ.to_qiskit().draw())

Circuit(num_gates: 15, num_qubits: 6, with_measure: False)
     ┌───┐┌───┐                         ┌───┐
q_0: ┤ H ├┤ Z ├──■──────────────────────┤ X ├
     └───┘└───┘┌─┴─┐                    └─┬─┘
q_1: ───────■──┤ X ├───────■──────────────┼──
     ┌───┐┌─┴─┐└───┘┌───┐┌─┴─┐            │  
q_2: ┤ H ├┤ X ├──■──┤ H ├┤ X ├────────────┼──
     └───┘└───┘┌─┴─┐├───┤└───┘┌───┐┌───┐  │  
q_3: ──────────┤ X ├┤ H ├──■──┤ Z ├┤ X ├──■──
               └───┘└───┘  │  └───┘└─┬─┘     
q_4: ───────■──────────────┼─────────■───────
     ┌───┐┌─┴─┐          ┌─┴─┐               
q_5: ┤ H ├┤ X ├──────────┤ X ├───────────────
     └───┘└───┘          └───┘               


In [12]:
circ_with_swaps.to_qiskit().draw()

┌───┐┌───┐                            ┌───┐
q_0: ┤ H ├┤ Z ├──■──────────────────X──────┤ X ├
     └───┘└───┘┌─┴─┐                │      └─┬─┘
q_1: ───────■──┤ X ├───────■────────┼────────┼──
     ┌───┐┌─┴─┐└───┘┌───┐┌─┴─┐      │        │  
q_2: ┤ H ├┤ X ├──■──┤ H ├┤ X ├──────┼────────┼──
     └───┘└───┘┌─┴─┐├───┤└───┘┌───┐ │ ┌───┐  │  
q_3: ──────────┤ X ├┤ H ├──■──┤ Z ├─X─┤ X ├──■──
               └───┘└───┘  │  └───┘   └─┬─┘     
q_4: ───────■──────────────┼────────────■───────
     ┌───┐┌─┴─┐          ┌─┴─┐                  
q_5: ┤ H ├┤ X ├──────────┤ X ├──────────────────
     └───┘└───┘          └───┘

In [13]:
mapped_circ = arch.unify_mapped_circuit(circ_with_swaps, [mapping, updated_mapping])
print(mapped_circ)
print(mapped_circ.to_qiskit().draw())

Circuit(num_gates: 18, num_qubits: 6, with_measure: False)
     ┌───┐┌───┐                    ┌───┐     ┌───┐┌───┐     
q_0: ┤ H ├┤ Z ├──■─────────────────┤ X ├──■──┤ X ├┤ X ├──■──
     └───┘└───┘┌─┴─┐               └─┬─┘  │  └─┬─┘└─┬─┘  │  
q_1: ───────■──┤ X ├───────■─────────┼────┼────┼────┼────┼──
     ┌───┐┌─┴─┐└───┘┌───┐┌─┴─┐       │    │    │    │    │  
q_2: ┤ H ├┤ X ├──■──┤ H ├┤ X ├───────┼────┼────┼────┼────┼──
     └───┘└───┘┌─┴─┐├───┤└───┘┌───┐  │  ┌─┴─┐  │    │  ┌─┴─┐
q_3: ──────────┤ X ├┤ H ├──■──┤ Z ├──■──┤ X ├──■────┼──┤ X ├
               └───┘└───┘  │  └───┘     └───┘       │  └───┘
q_4: ───────■──────────────┼────────────────────────■───────
     ┌───┐┌─┴─┐          ┌─┴─┐                              
q_5: ┤ H ├┤ X ├──────────┤ X ├──────────────────────────────
     └───┘└───┘          └───┘                              


In [14]:
arch.verify_mapped_circuit(circ, mapped_circ, mapping, updated_mapping)

True